# Part 3: Clustering for Neighborhoods of Toronto.
### In this section I will Explore and cluster the neighborhoods in Toronto. 

In [1]:
# importing old libraries (also used in the two first parts)
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs

In [4]:
# importing new libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |        py36_1001         494 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.0               |             py_0          24 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         592 KB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py36_1001 conda-forge
    branca:  0.3.0-py_0      conda-forge
    folium:  0.5.0-py_0      conda-forge
    vincent: 0.4.4-py_1      conda-forge


altair-2.2.2         | 494 KB    | #######################

### Load the pandas dataframe (created in part 2 of the assignment)

In [7]:
df_toronto = pd.read_csv('toronto_part2.csv')
df_toronto.head(7)

,Unnamed: 0,Postcode,Borough,Neighborhood,Latitude,Longitude
0,0,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
1,1,M4H,East York,Thorncliffe Park,43.705369,-79.349372
2,2,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
3,3,M9W,Etobicoke,Northwest,43.706748,-79.594054
4,4,M9L,North York,Humber Summit,43.756303,-79.565963
5,5,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
6,6,M9N,York,Weston,43.706876,-79.518188


### Create a map of Toronto

In [17]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

### Create a new data frame with neighborhoods in boroughs that contain the word Toronto

In [18]:
# @hiddel_cell
CLIENT_ID = 'Y1KM0AFK4A1H5M5FE0VAOP0N5XBWF5AKMYLIN1TZSTLJQPTL' # Foursquare ID
CLIENT_SECRET = 'JXVMZ2U32SCJWZQIYDHOKYIDFXGTAVJC4DQV0IOWZ21XX4VF' # Foursquare Secret
VERSION = '20180605' # Foursquare API version



In [55]:
borough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
borough_data.head(7)

,Unnamed: 0,Postcode,Borough,Neighborhood,Latitude,Longitude
0,2,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,11,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302
2,13,M1W,Scarborough,Steeles West,43.799525,-79.318389
3,32,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
4,33,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476
5,39,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
6,42,M1R,Scarborough,"Maryvale, Wexford",43.750071,-79.295849


### Create a map of Scarborough and its neighbourhoods

In [56]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [57]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(borough_data['Latitude'], borough_data['Longitude'], borough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

### Get the top 100 venues in the neighborhood 'Steeles West', from Scarborough

In [58]:
neighborhood_latitude = borough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = borough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = borough_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Rouge, Malvern" are 43.8066863, -79.19435340000003.


In [59]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [60]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bcda57bf594df1e00c4a225'},
 'response': {'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 43,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775537,
          'lng': -79.256833}],
        'distance

In [61]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [62]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,DAVIDsTEA,Tea Room,43.776613,-79.258516
2,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,SEPHORA,Cosmetics Shop,43.775592,-79.258242
5,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
6,Hot Topic,Clothing Store,43.775450,-79.257929
7,Scarborough Town Centre,Shopping Mall,43.775231,-79.257462
8,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
9,American Eagle Outfitters,Clothing Store,43.775908,-79.258352


In [63]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

43 venues were returned by Foursquare.


In [64]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get venues for each neighborhood

In [65]:
borough_venues = getNearbyVenues(names=borough_data['Neighborhood'],
                                   latitudes=borough_data['Latitude'],
                                   longitudes=borough_data['Longitude']
                                  )

Rouge, Malvern
Tam O'Shanter
Steeles West
Upper Rouge
Cliffcrest, Cliffside
Highland Creek, Rouge Hill, Port Union
Maryvale, Wexford
Scarborough Village
Agincourt North, Milliken
Clairlea, Golden Mile, Oakridge
Woburn
Ionview, Kennedy Park
Birch Cliff
Morningside, West Hill
Cedarbrae
Agincourt
Dorset Park, Scarborough Town Centre, Wexford Heights


In [66]:
borough_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Tam O'Shanter,43.781638,-79.304302,Remezzo Italian Bistro,43.778649,-79.308264,Italian Restaurant
3,Tam O'Shanter,43.781638,-79.304302,The Royal Chinese Restaurant 避風塘小炒,43.780505,-79.298844,Chinese Restaurant
4,Tam O'Shanter,43.781638,-79.304302,Eight Noodles,43.778234,-79.308299,Noodle House
5,Tam O'Shanter,43.781638,-79.304302,Kub Khao,43.780438,-79.299837,Thai Restaurant
6,Tam O'Shanter,43.781638,-79.304302,KFC,43.779440,-79.303371,Fast Food Restaurant
7,Tam O'Shanter,43.781638,-79.304302,Eaton Centre USA,43.783572,-79.304916,Shopping Mall
8,Tam O'Shanter,43.781638,-79.304302,Popeyes,43.780476,-79.298460,Fried Chicken Joint
9,Tam O'Shanter,43.781638,-79.304302,Little Caesars,43.780563,-79.298624,Pizza Place


In [67]:
borough_venues.tail(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
82,Agincourt,43.79420,-79.262029,Twilight,43.791999,-79.258584,Lounge
83,Agincourt,43.79420,-79.262029,Mark's,43.791179,-79.259714,Clothing Store
84,Agincourt,43.79420,-79.262029,Commander Arena,43.794867,-79.267989,Skating Rink
85,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Kim Kim restaurant,43.753833,-79.276611,Chinese Restaurant
86,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Kairali,43.754768,-79.277199,Indian Restaurant
87,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Karaikudi Chettinad South Indian Restaurant,43.756042,-79.276276,Indian Restaurant
88,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,El Pulgarcito,43.754790,-79.277064,Latin American Restaurant
89,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Big Al's Pet Supercentre,43.759279,-79.278325,Pet Store
90,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Pho Vietnam,43.757770,-79.278572,Vietnamese Restaurant
91,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Blinds To Go,43.757147,-79.278073,Furniture / Home Store


In [68]:
borough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Milliken",3,3,3,3,3,3
Birch Cliff,4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Cliffcrest, Cliffside",3,3,3,3,3,3
"Dorset Park, Scarborough Town Centre, Wexford Heights",7,7,7,7,7,7
"Highland Creek, Rouge Hill, Port Union",3,3,3,3,3,3
"Ionview, Kennedy Park",7,7,7,7,7,7


In [69]:
print('There are {} uniques categories.'.format(len(borough_venues['Venue Category'].unique())))

There are 58 uniques categories.


In [70]:
# one hot encoding
scarb_onehot = pd.get_dummies(borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = borough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Hakka Restaurant,History Museum,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Train Station,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Tam O'Shanter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Tam O'Shanter,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Tam O'Shanter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [71]:
scarb_onehot.shape

(92, 59)

In [72]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Hakka Restaurant,History Museum,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.0,0.000000,0.000,0.0,0.25,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.25,0.000000,0.00,0.0,0.0,0.0,0.0,0.000,0.000000,0.00,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.250,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.25,0.000000,0.0,0.000,0.0,0.000000
1,"Agincourt North, Milliken",0.000000,0.000,0.0,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.00,0.333333,0.00,0.0,0.0,0.0,0.0,0.000,0.000000,0.00,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.333333,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000,0.0,0.000000
2,Birch Cliff,0.000000,0.000,0.0,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.0,0.25,0.000,0.000000,0.00,0.000000,0.25,0.0,0.0,0.0,0.0,0.000,0.000000,0.25,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.25,0.000000,0.0,0.000,0.0,0.000000
3,Cedarbrae,0.000000,0.125,0.0,0.125000,0.125,0.0,0.00,0.000000,0.000000,0.0,0.00,0.125,0.000000,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.125,0.000000,0.00,0.0,0.125,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.125,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.125,0.0,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.000000,0.000,0.0,0.222222,0.000,0.0,0.00,0.222222,0.111111,0.0,0.00,0.000,0.000000,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.000,0.000000,0.00,0.0,0.000,0.0,0.0,0.111111,0.000000,0.111111,0.0,0.0,0.0,0.000000,0.000,0.0,0.111111,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.111111,0.0,0.000,0.0,0.000000
5,"Cliffcrest, Cliffside",0.333333,0.000,0.0,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.000,0.000000,0.00,0.0,0.000,0.0,0.0,0.000000,0.000000,0.333333,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000,0.0,0.000000
6,"Dorset Park, Scarborough Town Centre, Wexford ...",0.000000,0.000,0.0,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.0,0.00,0.000,0.142857,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.000,0.142857,0.00,0.0,0.000,0.0,0.0,0.000000,0.285714,0.000000,0.0,0.0,0.0,0.142857,0.000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.142857,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000,0.0,0.142857


### Get top 10 venues per neighborhood

In [73]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Coffee Shop,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store
1,"Agincourt North, Milliken",Playground,Park,Coffee Shop,Vietnamese Restaurant,Clothing Store,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store
2,Birch Cliff,Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,History Museum,Hakka Restaurant,Grocery Store,Furniture / Home Store
3,Cedarbrae,Lounge,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Electronics Store
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Ice Cream Shop,Intersection,Soccer Field,Metro Station,Bus Station,Furniture / Home Store,Electronics Store,Fast Food Restaurant
5,"Cliffcrest, Cliffside",American Restaurant,Intersection,Motel,Ice Cream Shop,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
6,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Pet Store,Chinese Restaurant,Furniture / Home Store,Latin American Restaurant,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Grocery Store,General Entertainment
7,"Highland Creek, Rouge Hill, Port Union",History Museum,Bar,Moving Target,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
8,"Ionview, Kennedy Park",Discount Store,Hobby Shop,Bus Station,Department Store,Train Station,Coffee Shop,Vietnamese Restaurant,College Stadium,History Museum,Hakka Restaurant
9,"Maryvale, Wexford",Middle Eastern Restaurant,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Fried Chicken Joint,Department Store,Discount Store,Electronics Store


### Run k-means to cluster the neighborhoods into 5 clusters

In [75]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = borough_data.drop(16)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([2, 4, 2, 2, 1, 1, 2, 3, 2, 2], dtype=int32)

### Include kmeans.labels_ into the original Scarborough dataframe

In [76]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarb_merged

,Unnamed: 0,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Clothing Store,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
1,11,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302,4,Pizza Place,Fried Chicken Joint,Italian Restaurant,Noodle House,Fast Food Restaurant,Rental Car Location,Chinese Restaurant,Shopping Mall,Thai Restaurant,Furniture / Home Store
2,13,M1W,Scarborough,Steeles West,43.799525,-79.318389,2,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Japanese Restaurant,Sandwich Place,Breakfast Spot,Coffee Shop,Pizza Place,Pharmacy,Nail Salon
3,32,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,33,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476,1,American Restaurant,Intersection,Motel,Ice Cream Shop,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
5,39,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1,History Museum,Bar,Moving Target,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
6,42,M1R,Scarborough,"Maryvale, Wexford",43.750071,-79.295849,2,Middle Eastern Restaurant,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Fried Chicken Joint,Department Store,Discount Store,Electronics Store
7,47,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,3,Spa,Playground,Business Service,Vietnamese Restaurant,Coffee Shop,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store
8,49,M1V,Scarborough,"Agincourt North, Milliken",43.815252,-79.284577,2,Playground,Park,Coffee Shop,Vietnamese Restaurant,Clothing Store,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store
9,50,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,2,Bakery,Bus Line,Ice Cream Shop,Intersection,Soccer Field,Metro Station,Bus Station,Furniture / Home Store,Electronics Store,Fast Food Restaurant


### Visualize the clusters in the map

In [78]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighborhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine each of the five clusters

In [79]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,M1K,-79.262029,0,Discount Store,Hobby Shop,Bus Station,Department Store,Train Station,Coffee Shop,Vietnamese Restaurant,College Stadium,History Museum,Hakka Restaurant


In [80]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M1M,-79.239476,1,American Restaurant,Intersection,Motel,Ice Cream Shop,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
5,M1C,-79.160497,1,History Museum,Bar,Moving Target,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint


In [81]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]    

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,-79.194353,2,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Clothing Store,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
2,M1W,-79.318389,2,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Japanese Restaurant,Sandwich Place,Breakfast Spot,Coffee Shop,Pizza Place,Pharmacy,Nail Salon
3,M1X,-79.205636,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1R,-79.295849,2,Middle Eastern Restaurant,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Fried Chicken Joint,Department Store,Discount Store,Electronics Store
8,M1V,-79.284577,2,Playground,Park,Coffee Shop,Vietnamese Restaurant,Clothing Store,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store
9,M1L,-79.284577,2,Bakery,Bus Line,Ice Cream Shop,Intersection,Soccer Field,Metro Station,Bus Station,Furniture / Home Store,Electronics Store,Fast Food Restaurant
10,M1G,-79.216917,2,Coffee Shop,Indian Restaurant,Korean Restaurant,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
12,M1N,-79.264848,2,Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,History Museum,Hakka Restaurant,Grocery Store,Furniture / Home Store
13,M1E,-79.188711,2,Rental Car Location,Mexican Restaurant,Medical Center,Pizza Place,Breakfast Spot,Electronics Store,Fast Food Restaurant,Department Store,Discount Store,Vietnamese Restaurant
14,M1H,-79.239476,2,Lounge,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Electronics Store


In [82]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,M1J,-79.239476,3,Spa,Playground,Business Service,Vietnamese Restaurant,Coffee Shop,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store


In [83]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1T,-79.304302,4,Pizza Place,Fried Chicken Joint,Italian Restaurant,Noodle House,Fast Food Restaurant,Rental Car Location,Chinese Restaurant,Shopping Mall,Thai Restaurant,Furniture / Home Store
15,M1S,-79.262029,4,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Coffee Shop,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store


-----------------end of Part 3 ------------------------